# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **lab12** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team name**: Par de Dos

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Recommender-Systems") \
    .master("spark://66728873c271:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/11 19:48:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Preparación de Datos

In [ ]:
from team_name.spark_utils import SparkUtils
from team_ParDeDos.spark_utils import SparkUtils as SpU
from pyspark.sql.functions import col, split

schema = SpU.generate_schema([
        ("user_id", "integer"),
        ("song_id", "integer"),
        ("rating", "integer"),
        ("timestamp", "long")
    ])

data = spark.read \
    .option("header","true") \
    .text("/home/jovyan/notebooks/data/sample_movielens_ratings.txt")

data = data.select(
    split(col("value"), "::").getItem(0).cast("integer").alias("user_id"),
    split(col("value"), "::").getItem(1).cast("integer").alias("movie_id"),
    split(col("value"), "::").getItem(2).cast("integer").alias("rating"),
    split(col("value"), "::").getItem(3).cast("long").alias("timestamp")
)

data.show()

data.printSchema()

+-------+--------+------+----------+
|user_id|movie_id|rating| timestamp|
+-------+--------+------+----------+
|      0|       2|     3|1424380312|
|      0|       3|     1|1424380312|
|      0|       5|     2|1424380312|
|      0|       9|     4|1424380312|
|      0|      11|     1|1424380312|
|      0|      12|     2|1424380312|
|      0|      15|     1|1424380312|
|      0|      17|     1|1424380312|
|      0|      19|     1|1424380312|
|      0|      21|     1|1424380312|
|      0|      23|     1|1424380312|
|      0|      26|     3|1424380312|
|      0|      27|     1|1424380312|
|      0|      28|     1|1424380312|
|      0|      29|     1|1424380312|
|      0|      30|     1|1424380312|
|      0|      31|     1|1424380312|
|      0|      34|     1|1424380312|
|      0|      37|     1|1424380312|
|      0|      41|     2|1424380312|
+-------+--------+------+----------+
only showing top 20 rows

root
 |-- user_id: integer (nullable = true)
 |-- movie_id: integer (nullable = true)


# Configure ALS model

In [5]:
from pyspark.ml.recommendation import ALS

als = ALS(
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    maxIter=10,
    regParam=0.1,
    rank=5,
    coldStartStrategy="drop"
)

# TRAINNING

In [6]:
model = als.fit(data)

# PREDICTIONS

In [7]:
user_recommendations = model.recommendForAllUsers(numItems=5)

user_recommendations.show(truncate=False)

+-------+-------------------------------------------------------------------------------------+
|user_id|recommendations                                                                      |
+-------+-------------------------------------------------------------------------------------+
|0      |[{92, 2.5840385}, {2, 2.316802}, {62, 2.2325232}, {25, 2.157748}, {93, 2.1528697}]   |
|10     |[{92, 2.768342}, {2, 2.6728113}, {93, 2.6242015}, {25, 2.5927775}, {49, 2.5867324}]  |
|20     |[{22, 3.5597918}, {68, 3.1278815}, {94, 3.084497}, {51, 3.0827737}, {77, 3.0246763}] |
|1      |[{22, 2.9029422}, {68, 2.630123}, {77, 2.5238972}, {62, 2.501064}, {90, 2.4797387}]  |
|11     |[{32, 5.082464}, {18, 4.705235}, {30, 4.6826043}, {27, 4.5120797}, {8, 4.229401}]    |
|21     |[{29, 4.320379}, {52, 4.2401457}, {76, 3.716108}, {63, 3.5063725}, {53, 3.4859684}]  |
|22     |[{51, 4.458179}, {75, 4.418395}, {22, 4.118836}, {74, 4.1007586}, {88, 4.0829244}]   |
|2      |[{93, 4.2531066}, {83, 4.146952

## Predictions for all data

In [8]:
predictions = model.transform(data)
predictions.show(truncate=False)

+-------+--------+------+----------+----------+
|user_id|movie_id|rating|timestamp |prediction|
+-------+--------+------+----------+----------+
|22     |0       |1     |1424380312|0.96697557|
|22     |3       |2     |1424380312|1.6326257 |
|22     |5       |2     |1424380312|2.0366673 |
|22     |6       |2     |1424380312|2.2972772 |
|22     |9       |1     |1424380312|1.5513803 |
|22     |10      |1     |1424380312|1.4349127 |
|22     |11      |1     |1424380312|1.2901659 |
|22     |13      |1     |1424380312|1.617328  |
|22     |14      |1     |1424380312|1.389045  |
|22     |16      |1     |1424380312|0.7093756 |
|22     |18      |3     |1424380312|3.0116072 |
|22     |19      |1     |1424380312|1.4644071 |
|22     |22      |5     |1424380312|4.118836  |
|22     |25      |1     |1424380312|0.97840077|
|22     |26      |1     |1424380312|1.1323681 |
|22     |29      |3     |1424380312|3.2431226 |
|22     |30      |5     |1424380312|3.9942718 |
|22     |32      |4     |1424380312|3.21

# EVALUATE MODEL

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error (RMSE) = {rmse}")

Root-mean-square error (RMSE) = 0.5691166521341573


In [10]:
sc.stop()